<a href="https://colab.research.google.com/github/cheol-jung/chatbot/blob/main/find_tune_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#from google.colab import drive
#drive.mount('/content/drive')

#!pip install transformers datasets

In [22]:
import pandas as pd
from datasets import Dataset

# CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/dataset/qa_dataset_v2.csv')

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)

In [23]:
from transformers import AutoTokenizer

# T5 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('t5-base')

# 입력 텍스트 포맷: 질문과 스키마를 하나로 묶음
def preprocess_function(examples):
    inputs = [f"Translate the question: {question} using schema: {schema}" for question, schema in zip(examples['question'], examples['db_schema'])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length') # Add padding to the tokenizer

    # 정답(타겟): SQL 쿼리
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], max_length=512, truncation=True, padding='max_length') # Add padding to the tokenizer

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# 데이터셋 토큰화
tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [26]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# 모델 로드 (사전학습된 T5 모델)
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# 훈련 설정
training_args = TrainingArguments(
    output_dir='./results',          # 모델 결과 저장 경로
    evaluation_strategy="epoch",     # 에포크마다 평가
    learning_rate=2e-5,              # 학습률
    per_device_train_batch_size=8,   # 배치 사이즈
    per_device_eval_batch_size=8,
    num_train_epochs=100,              # 학습 에포크 수
    weight_decay=0.01,               # 가중치 감쇠
    save_total_limit=2,              # 저장할 체크포인트 수 제한
    save_steps=500,                  # 체크포인트 저장 주기
    eval_steps=500,                  # 평가 주기
    logging_dir='./logs',            # 로그 저장 경로
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # 훈련 데이터셋
    eval_dataset=tokenized_dataset,   # 평가 데이터셋
)

# 모델 훈련 시작
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,23.785015
2,No log,21.473457
3,No log,17.898460
4,No log,13.940611
5,No log,12.618054
6,No log,12.187947
7,No log,11.792217
8,No log,11.190789
9,No log,10.543044
10,No log,9.882552


TrainOutput(global_step=1000, training_loss=0.27822013694047926, metrics={'train_runtime': 1543.6437, 'train_samples_per_second': 2.591, 'train_steps_per_second': 0.648, 'total_flos': 2435831562240000.0, 'train_loss': 0.27822013694047926, 'epoch': 1000.0})

In [27]:
# 평가
trainer.evaluate()

def generate_sql(question, schema):
    input_text = f"Translate the question: {question} using schema: {schema}"
    inputs = tokenizer(input_text, return_tensors='pt', max_length=512, truncation=True)

    # Move inputs to the same device as the model
    inputs = inputs.to(model.device)

    # 모델 예측
    outputs = model.generate(input_ids=inputs['input_ids'], max_length=512)
    # 결과를 텍스트로 변환
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_sql

# 예시 질문과 스키마
new_question = "2023년에 발생한 모든 판매의 총합을 구해줘."
new_schema = "sales(date DATE, amount DECIMAL(10, 2), product_id INT)"

# 모델이 생성한 SQL 쿼리 출력
generated_sql = generate_sql(new_question, new_schema)
print("Generated SQL:", generated_sql)

Generated SQL: SELECT SUM(amount) FROM sales WHERE YEAR(date) = 2023;
